In [1]:
import os
import sys

import django

sys.path.append('../')  # add path to project root dir

os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

  * Reading config from flourish.ini
Loading Data Encryption (init)...
 * loading keys from /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields
 * loading rsa.restricted.public ... Done.
 * loading rsa.restricted.private ... Done.
 * loading rsa.local.public ... Done.
 * loading rsa.local.private ... Done.
 * loading aes.local ... Done.
 * loading aes.restricted ... Done.
 * loading salt.local ... Done.
 * loading salt.restricted ... Done.
 Done loading Data Encryption (init)...


/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/django_q/conf.py:139: UserWarning: Retry and timeout are misconfigured. Set retry larger than timeout, 
        failure to do so will cause the tasks to be retriggered before completion. 
        See https://django-q.readthedocs.io/en/latest/configure.html#retry for details.
  warn(
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (HistoryManagerMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (ScheduleModelMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (VisitModelMixin) without inherit=True
  warnings.wa

Loading Data Encryption ...
 * found encryption keys in /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields.
 * using model django_crypto_fields.crypt.
 Done loading Data Encryption.
Loading Edc Consent ...
 * checking for site consents ...
 * registered consents 'consents' from 'flourish_caregiver'
 * registered consents 'consents' from 'flourish_child'
 * registered consents 'consents' from 'pre_flourish'
 * registered consents 'consents' from 'flourish_facet'
 * flourish_caregiver.subjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 2 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 3 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 4 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbadolconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbinformedconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.childdum

In [67]:
from edc_appointment.models import Appointment
import datetime
from edc_visit_schedule import site_visit_schedules
from edc_visit_schedule.models import SubjectScheduleHistory
from django.apps import apps as django_apps
import pytz

def localised_time(date_str):
    format_str = "%Y/%m/%d"
    naive_date = datetime.datetime.strptime(date_str, format_str)

    utc_date = pytz.utc.localize(naive_date)

    gaborone = pytz.timezone('Africa/Gaborone')
    gaborone_date = utc_date.astimezone(gaborone)
    return gaborone_date

subject_identifier = 'B142-040990894-3'
subject_schedule_history_objs = SubjectScheduleHistory.objects.filter(
    subject_identifier=subject_identifier)
schedule_name = 'b_fu_quarterly1_schedule1'
onschedule_model = 'flourish_caregiver.onschedulecohortbfuquarterly'
_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model=onschedule_model,
    name=schedule_name)
child_schedule_history = SubjectScheduleHistory.objects.get(
    subject_identifier='B142-040990894-3-10', schedule_name='child_b_fu_qt_schedule1')

schedule.put_on_schedule(
    subject_identifier=subject_identifier,
    onschedule_datetime=child_schedule_history.onschedule_datetime,
    base_appt_datetime=localised_time('2023/03/09'),
    schedule_name=schedule_name)
    
onschedule_obj = schedule.onschedule_model_cls.objects.get(
    subject_identifier=subject_identifier,
    schedule_name=schedule_name)

onschedule_obj.child_subject_identifier = child_schedule_history.subject_identifier
onschedule_obj.save()

caregiver_3000_apps = Appointment.objects.filter(
    subject_identifier=subject_identifier,
    schedule_name='b_fu_quarterly1_schedule1',
    visit_code__in=['3001M','3002M', '3003M', '3004M']
)

for ap in caregiver_3000_apps:
    ap.delete()

onschedule_model_cls = django_apps.get_model('flourish_caregiver.onschedulecohortbsecseq')
onschedule_model_obj = onschedule_model_cls.objects.get(
    subject_identifier=subject_identifier)
onschedule_model_obj.delete()
subject_schedule_history_obj = SubjectScheduleHistory.objects.get(
    subject_identifier=subject_identifier, schedule_name='b_sec_fu_quart1_schedule1')
subject_schedule_history_obj.delete()

caregiver_sec_aps=Appointment.objects.filter(
    subject_identifier=subject_identifier,
    schedule_name='b_sec_fu_quart1_schedule1',
)

for ap in caregiver_sec_aps:
    ap.delete()